# Postprocessing

In [1]:
import re
import os
import pandas as pd

## Read validation data

In [2]:
def read_validation_directory():
    available_content = os.listdir('.') 
    select_pattern = re.compile('^validation-[a-z][a-z][a-z]')
    selected_dir = [content for content in available_content if select_pattern.match(content)]
    return selected_dir

In [3]:
def read_validation_directory_content(validation_directories):
    iteration = 0
    for directory in validation_directories:
        available_content = os.listdir(directory)
        for file in available_content:
            if iteration == 0:
                imported_data = pd.read_csv('{}/{}'.format(directory, file))
                imported_data['model'] = directory[11:]
            else:
                tmp_data = pd.read_csv('{}/{}'.format(directory, file)) 
                tmp_data['model'] = directory[11:] 
                imported_data = pd.concat([imported_data, tmp_data])
            iteration += 1
    
    imported_data.reset_index(drop=True, inplace=True)
    return imported_data

In [4]:
def text_split(x):
    y = x.split('-')
    return y

In [5]:
def extract_feature_information(df):
    
    df_splitted = df['data'].apply(text_split)
    
    feature_method_list = []
    document_size_list = []
    feature_size_list = []
    ngram_size_list = []
    
    for splitted_entries in df_splitted:
        if splitted_entries[1] == 'idf': # inverse document frequency case
            feature_method_list.append('tf-idf')
            document_size_list.append(splitted_entries[2])
            feature_size_list.append(splitted_entries[3])
            ngram_size_list.append(splitted_entries[4])
        else:
            feature_method_list.append('tf')
            document_size_list.append(splitted_entries[1])
            feature_size_list.append(splitted_entries[2])
            ngram_size_list.append(splitted_entries[3])
    
    df['method'] = feature_method_list
    df['doc'] = document_size_list
    df['feature'] = feature_size_list
    df['ngram'] = ngram_size_list
    df.drop(['data'], axis=1, inplace=True)
    
    # set index
    df.index = df['model'] + '-' + df['method'] + '-' + df['doc'] + '-' + df['feature'] + '-' + df['ngram']
    
    return df

In [6]:
def to_percent(x):
    return round(x * 100, 2)

In [7]:
def read_results():
    validation_directories = read_validation_directory()
    raw_results = read_validation_directory_content(validation_directories)
    results = extract_feature_information(raw_results)
    results.sort_values(by=['validation_accuracy'], ascending=False, inplace=True)
    results['validation_accuracy'] = to_percent(results['validation_accuracy'])
    results['test_accuracy'] = to_percent(results['test_accuracy'])
    return results

In [8]:
# read from all validation directories the accuracy and determine model settings
results = read_results()
results.head(results.shape[0]) # display all

,validation_accuracy,test_accuracy,model,method,doc,feature,ngram
svc-tf-idf-5000-1000-1,97.36,96.02,svc,tf-idf,5000,1000,1
svc-tf-5000-5000-1,97.05,96.13,svc,tf,5000,5000,1
svc-tf-5000-1000-1,96.54,95.31,svc,tf,5000,1000,1
svc-tf-idf-5000-5000-1,96.03,95.21,svc,tf-idf,5000,5000,1
svc-tf-idf-5000-500-1,95.73,95.41,svc,tf-idf,5000,500,1
svc-tf-5000-500-1,95.42,95.11,svc,tf,5000,500,1
log-tf-5000-500-1,93.18,91.95,log,tf,5000,500,1
knn-tf-idf-5000-1000-1,92.07,92.25,knn,tf-idf,5000,1000,1
knn-tf-5000-1000-1,91.56,92.66,knn,tf,5000,1000,1
knn-tf-5000-500-1,91.15,92.35,knn,tf,5000,500,1


## Read tuned models

In [9]:
def read_tuned_model_directory_content(prefix):
    path = 'tuned-model-{}'.format(prefix)
    available_content = os.listdir(path)
    iteration = 0
    for file in available_content:
        if iteration == 0:
            imported_data = pd.read_csv('{}/{}'.format(path, file))
            imported_data['model'] = prefix
        else:
            tmp_data = pd.read_csv('{}/{}'.format(path, file)) 
            tmp_data['model'] = prefix
            imported_data = pd.concat([imported_data, tmp_data])
        iteration += 1   
            
    imported_data.reset_index(drop=True, inplace=True)
    return imported_data    

In [10]:
def read_tuned_model_results(prefix):
    raw_results = read_tuned_model_directory_content(prefix)
    results = extract_feature_information(raw_results)
    return results

In [11]:
def join_with_accuracy(tuned_model_results, accuracy_results):
    accuracy_drop = accuracy_results.drop(['model', 'method', 'doc', 'feature', 'ngram'], axis=1)
    tuned_model_results = tuned_model_results.join(accuracy_drop, how='inner')
    tuned_model_results.sort_values(by=['validation_accuracy'], ascending=False, inplace=True)
    tuned_model_results.reset_index(drop=True, inplace=True)
    return tuned_model_results

In [12]:
def evaluate_average_performance(df, group_variable):
    performance = df[['validation_accuracy', group_variable]].groupby([group_variable]).mean()
    performance.sort_values(by=['validation_accuracy'], ascending=False, inplace=True)
    return performance

## k nearest neighbors model

In [13]:
results_knn = read_tuned_model_results('knn')
results_knn = join_with_accuracy(results_knn, results)
results_knn.head(results_knn.shape[0]) # display all

,p,weights,n_neighbors,model,method,doc,feature,ngram,validation_accuracy,test_accuracy
0,1,uniform,13,knn,tf-idf,5000,1000,1,92.07,92.25
1,1,uniform,13,knn,tf,5000,1000,1,91.56,92.66
2,1,distance,8,knn,tf,5000,500,1,91.15,92.35
3,1,distance,12,knn,tf-idf,5000,500,1,90.74,91.95
4,1,uniform,15,knn,tf,5000,5000,1,89.83,91.13
5,6,distance,4,knn,tf,5000,1000,2,79.45,81.35
6,1,distance,6,knn,tf-idf,5000,5000,1,79.25,79.41
7,7,distance,7,knn,tf,5000,500,2,79.25,81.24
8,6,distance,3,knn,tf,5000,5000,2,79.04,80.84
9,8,distance,3,knn,tf-idf,5000,1000,2,78.03,78.90


## Support vector models

In [14]:
results_svc = read_tuned_model_results('svc')
results_svc = join_with_accuracy(results_svc, results)
results_svc.head(results_svc.shape[0]) # display all

,random_state,degree,kernel,C,model,method,doc,feature,ngram,validation_accuracy,test_accuracy
0,1,2,poly,2.953042,svc,tf-idf,5000,1000,1,97.36,96.02
1,1,2,poly,2.815403,svc,tf,5000,5000,1,97.05,96.13
2,1,2,poly,1.097294,svc,tf,5000,1000,1,96.54,95.31
3,1,2,poly,3.000000,svc,tf-idf,5000,5000,1,96.03,95.21
4,1,2,poly,2.811214,svc,tf-idf,5000,500,1,95.73,95.41
5,1,2,poly,3.000000,svc,tf,5000,500,1,95.42,95.11
6,1,2,poly,3.000000,svc,tf,5000,500,2,90.13,88.58
7,1,2,poly,2.839080,svc,tf,5000,1000,2,89.73,90.01
8,1,2,poly,2.449617,svc,tf-idf,5000,1000,2,89.22,89.40
9,1,2,poly,2.973663,svc,tf-idf,5000,5000,2,89.11,88.48


## Logistic regression model

In [15]:
results_log = read_tuned_model_results('log')
results_log = join_with_accuracy(results_log, results)
results_log.head(results_log.shape[0]) # display all

,C,max_iter,model,method,doc,feature,ngram,validation_accuracy,test_accuracy
0,2.992811,10000,log,tf,5000,500,1,93.18,91.95


## Model performance

In [16]:
model_performance = evaluate_average_performance(results, 'model')
model_performance

,validation_accuracy
model,
log,93.180000
svc,92.743333
knn,83.640000


## Feature number performance

In [17]:
feature_performance = evaluate_average_performance(results, 'feature')
feature_performance

,validation_accuracy
feature,
1000,89.245000
500,89.103333
5000,86.736250


## Feature method performance

In [18]:
method_performance = evaluate_average_performance(results, 'method')
method_performance

,validation_accuracy
method,
tf,89.240000
tf-idf,87.471667


## n-gram performance

In [19]:
ngram_performance = evaluate_average_performance(results, 'ngram')
ngram_performance

,validation_accuracy
ngram,
1,92.762308
2,83.655833
